In [1]:
# from models import xresnet1d,fastai_model
import torch
import torch.nn as nn
from configs.fastai_configs import *
import pandas as pd
import wfdb
import skimage
from torch.utils.data import DataLoader
# from experiments.scp_experiment import SCP_Experiment
import numpy as np
from tqdm import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, Flatten, Dense, BatchNormalization
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from tensorflow.keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
import seaborn as sns
import wfdb
import wfdb.processing
import ast
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import scipy
from tqdm import tqdm
import neurokit2 as nk

In [2]:
df = pd.read_csv("/Users/nisssaarg/Desktop/ecg_ptbxl_benchmarking-master/data/ptbxl/ptbxl.csv")

In [ ]:
df = df[df['CD']==1]

In [3]:
df

,Unnamed: 0,filename_hr,age,sex,CD,HYP,MI,NORM,STTC
0,0,records500/00000/00001_hr,56.0,1,0,0,0,1,0
1,1,records500/00000/00002_hr,19.0,0,0,0,0,1,0
2,2,records500/00000/00003_hr,37.0,1,0,0,0,1,0
3,3,records500/00000/00004_hr,24.0,0,0,0,0,1,0
4,4,records500/00000/00005_hr,19.0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
21383,21383,records500/21000/21833_hr,67.0,1,0,0,0,0,1
21384,21384,records500/21000/21834_hr,300.0,0,0,0,0,1,0
21385,21385,records500/21000/21835_hr,59.0,1,0,0,0,0,1
21386,21386,records500/21000/21836_hr,64.0,1,0,0,0,1,0


In [4]:
path = '/Users/nisssaarg/Desktop/ecg_ptbxl_benchmarking-master/data/ptbxl/'
sampling_rate=500

In [ ]:
#For multiclass
df = pd.read_csv("/Users/nisssaarg/Desktop/Deep Learning/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/ptbxl_database.csv")
data_ptbxl = df.drop(columns=['height','weight','nurse','site','device','recording_date','heart_axis','infarction_stadium1','infarction_stadium2','validated_by','second_opinion','initial_autogenerated_report','validated_by_human','baseline_drift','baseline_drift','static_noise','burst_noise','electrodes_problems','electrodes_problems','extra_beats','pacemaker','strat_fold'],axis=1)
data_statements = pd.read_csv("/Users/nisssaarg/Desktop/Deep Learning/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/scp_statements.csv")
path = '/Users/nisssaarg/Desktop/Deep Learning/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/'
sampling_rate=500

# load and convert annotation data
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))


Y= Y.explode("scp_codes")
scp = data_statements.drop(['description','diagnostic','form','rhythm','diagnostic_subclass','Statement Category','SCP-ECG Statement Description','AHA code','aECG REFID','DICOM Code','CDISC Code'],axis=1)
scp.columns=['scp_codes','diagnostic_class']
Y = Y.replace(scp.set_index('scp_codes')['diagnostic_class'])
Norm = Y[Y['scp_codes']=='NORM']
cd = Y[Y['scp_codes']=='CD']
mi = Y[Y['scp_codes']=='MI']
hyp = Y[Y['scp_codes']=='HYP']
sttc = Y[Y['scp_codes']=='STTC']

Y = pd.concat([Norm,cd,mi,hyp,sttc],axis=0)

Y = Y.drop_duplicates(subset='filename_hr')



In [5]:
df= df[(df['sex']==0)&(df['age']>65)]

In [6]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in tqdm(df.filename_hr)]
    data = np.array([signal for signal, meta in data])
    return data
X  = load_raw_data(df,sampling_rate,path)

100%|██████████| 3951/3951 [00:06<00:00, 579.11it/s]


In [ ]:
def categorize_age(age):
    if age<65:
        return 'young'
    else:
        return 'old'

In [ ]:
df['age_group'] = df['age'].apply(categorize_age)

In [9]:
Y = np.array(df[['CD','HYP','MI','NORM','STTC']])


In [ ]:
Y = df["age_group"]

In [ ]:
from models.rnn1d import RNN1d

In [63]:
from models.fastai_model import fastai_model

experiment = 'exp1.1.1'
modelname = 'lstm'
pretrainedfolder = '../output/'+experiment+'/models/'+modelname+'/'
mpath='../output/' # <=== path where the finetuned model will be stored
n_classes_pretrained =5 # <=== because we load the model from exp0, this should be fixed because this depends the experiment
num_classes = 2
sampling_frequency = 500
input_shape = (5000,12)
model = fastai_model(
    modelname, 
    num_classes, 
    sampling_frequency, 
    mpath, 
    input_shape=input_shape, 
    # pretrainedfolder=pretrainedfolder,
    # n_classes_pretrained=n_classes_pretrained, 
    # pretrained=True,
    # epochs_finetuning=2,
    epochs=0
)

In [64]:
from models.fastai_model import fastai_model

experiment = 'exp1.1.1'
modelname = 'conf_fastai_lstm'
pretrainedfolder = '../output/'+experiment+'/models/'+modelname+'/'
mpath='../output/' # <=== path where the finetuned model will be stored
n_classes_pretrained =5 # <=== because we load the model from exp0, this should be fixed because this depends the experiment
num_classes = 2
sampling_frequency = 500
input_shape = (5000,12)
model = fastai_model(
    modelname, 
    num_classes, 
    sampling_frequency, 
    mpath, 
    input_shape=input_shape, 
    # pretrainedfolder=pretrainedfolder,
    # n_classes_pretrained=n_classes_pretrained, 
    # pretrained=True,
    # epochs_finetuning=2,
    epochs=0
)

In [67]:
from models.rnn1d import RNN1d

experiment = 'exp1.1.1'
sampling_frequency=500
modelname = 'conf_fastai_lstm'
pretrainedfolder = '../output/'+experiment+'/models/'+modelname+'/'
mpath='../output/' # <=== path where the finetuned model will be stored
n_classes_pretrained =5 # <=== because we load the model from exp0, this should be fixed because this depends the experiment
num_classes = 5
input_shape = (5000,12)
model = RNN1d(
    modelname, 
    num_classes, 
    sampling_frequency, 
    mpath, 
    # input_shape=input_shape, 
    # pretrainedfolder=pretrainedfolder,
    # n_classes_pretrained=n_classes_pretrained, 
    # pretrained=True,
    # epochs_finetuning=2,
)

TypeError: hidden_size should be of type int, got: str

In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

NameError: name 'Y' is not defined

In [ ]:
def transform_label(Y):
    label_encoder = LabelEncoder()
    label_encoder.fit(Y)
    numerical_labels = label_encoder.transform(Y)
    decoded_strings = [(label_encoder.classes_[i] , i) for i in numerical_labels]

    one_hot_labels = to_categorical(numerical_labels)
    
    return np.array(one_hot_labels)

In [ ]:
Y_test

In [ ]:
import numpy as np
my_dict = {
    1: [],
    2: []
}

# Assume X_test and Y_test exist as numpy arrays
num_iterations = 20
test_size = len(X_test)

for i in range(num_iterations):
    indices = np.random.choice(test_size, size=int(test_size*0.8), replace=False)
    X_subset = X_test[indices]
    Y_subset = Y_test[indices]

    Y_pred = model.predict(X_subset)

    for j in range(2):    
        true_labels = Y_subset[:,j]
        predicted_probs = Y_pred[:,j]

        # Calculate the AUC score
        auc_score = roc_auc_score(true_labels, predicted_probs)
        my_dict[j+1].append(auc_score)
    
    

In [ ]:
cm_percent = cm / cm.sum(axis=1).reshape(-1, 1) * 100
sns.heatmap(cm_percent, annot=True, cmap="Blues")
plt.title("Confusion Matrix (Percentage)")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
my_dict = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: []
}
for i in range(20):
    X_test = X_test_splits[i]
    Y_test = Y_test_splits[i]
    
    Y_pred = model.predict(X_test)
    # True labels (ground truth) and predicted probabilities
    for j in range(5):    
        true_labels = Y_test[:,j]
        predicted_probs = Y_pred[:,j]

        # Calculate the AUC score
        auc_score = roc_auc_score(true_labels, predicted_probs)
        my_dict[j+1].append(auc_score)
        

In [ ]:
import csv
csv_filename = 'data.csv'

# Writing the dictionary to a CSV file
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=my_dict.keys())

    # Write header
    writer.writeheader()

    # Write rows
    for i in range(len(my_dict[1])):
        row = {key: my_dict[key][i] for key in my_dict.keys()}
        writer.writerow(row)

# Female old

In [ ]:
import matplotlib.pyplot as plt

# 'CD' data
data_cd = [
    [0.77799228, 0.85128205, 0.77799228, 0.91758242],
    [0.86363636, 0.91168831, 0.86363636, 0.85204082],
    [0.91666667, 0.94545455, 0.91666667, 0.92727273],
    [0.81632653, 0.91147541, 0.81632653, 0.92333333],
    [0.96055227, 0.92873563, 0.96055227, 0.9672619],
    [0.88669951, 0.98333333, 0.88669951, 0.86153846],
    [0.91025641, 0.91298701, 0.91025641, 0.90933333],
    [0.86941964, 0.97175141, 0.86941964, 0.92380952],
    [0.93269231, 0.92881356, 0.93269231, 0.83791209],
    [0.84693878, 0.98571429, 0.84693878, 0.85757576],
    [0.92460317, 0.92318059, 0.92460317, 0.86904762],
    [0.908867, 0.85402299, 0.908867, 0.92207792],
    [0.91563275, 0.94619883, 0.91563275, 0.66285714],
    [0.90909091, 0.85862069, 0.90909091, 0.94642857],
    [0.77346939, 0.9, 0.77346939, 0.87575758],
    [0.84722222, 0.92643678, 0.84722222, 0.93596059],
    [0.8982684, 0.88095238, 0.8982684, 0.95308642],
    [0.88095238, 0.95057471, 0.88095238, 0.88690476],
    [0.89661654, 0.92121212, 0.89661654, 0.85333333],
    [0.86706349, 0.96774194, 0.86706349, 0.89417989]
]

transposed_data_cd = list(zip(*data_cd))

# Plotting
plt.boxplot(transposed_data_cd, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('CD')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.75, 1.0)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 'HYP' data
data_hyp = [
    [0.82848837, 0.81355932, 0.82848837, 0.92207792],
    [0.87820513, 0.7295082, 0.87820513, 0.53198653],
    [0.75297619, 0.87298387, 0.75297619, 0.7887931],
    [0.79878049, 0.70588235, 0.79878049, 0.52040816],
    [0.71875, 0.84230769, 0.71875, 0.91666667],
    [0.58823529, 0.94246032, 0.58823529, 0.76190476],
    [0.61290323, 0.89754098, 0.61290323, 0.7745098],
    [0.93421053, 0.95833333, 0.93421053, 0.68421053],
    [0.69932432, 0.65909091, 0.69932432, 0.75757576],
    [0.54722222, 0.75793651, 0.54722222, 0.77956989],
    [0.58333333, 0.87923729, 0.58333333, 0.93464052],
    [0.91071429, 0.86346154, 0.91071429, 0.73888889],
    [0.80208333, 0.95703125, 0.80208333, 0.83333333],
    [0.75328947, 0.81076389, 0.75328947, 0.72395833],
    [0.63719512, 0.88319672, 0.63719512, 0.71505376],
    [0.8452381, 0.69038462, 0.8452381, 0.95495495],
    [0.48076923, 0.9265873, 0.48076923, 0.73161765],
    [0.83630952, 0.70384615, 0.83630952, 0.72916667],
    [0.73901809, 0.69153226, 0.73901809, 0.84803922],
    [0.83333333, 0.77124183, 0.83333333, 0.81904762]
]

transposed_data_hyp = list(zip(*data_hyp))

# Plotting
plt.boxplot(transposed_data_hyp, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('HYP')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.4, 1.0)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 'MI' data
data_mi = [
    [0.88803089, 0.85941043, 0.88803089, 0.83466667],
    [0.73529412, 0.89433551, 0.73529412, 0.80769231],
    [0.85119048, 0.95833333, 0.85119048, 0.74107143],
    [0.84693878, 0.90421456, 0.84693878, 0.59539474],
    [0.88362919, 0.89090909, 0.88362919, 0.76231884],
    [0.63793103, 0.85534591, 0.63793103, 0.6],
    [0.84975369, 0.916122, 0.84975369, 0.89066667],
    [0.91375291, 0.86111111, 0.91375291, 0.71651786],
    [0.80875576, 0.86111111, 0.80875576, 0.75466667],
    [0.83289125, 0.90958606, 0.83289125, 0.60909091],
    [0.7281746, 0.85714286, 0.7281746, 0.82619048],
    [0.68461538, 0.9012605, 0.68461538, 0.85079365],
    [0.85714286, 0.94117647, 0.85714286, 0.8],
    [0.66964286, 0.85504202, 0.66964286, 0.82898551],
    [0.81836735, 0.91721133, 0.81836735, 0.68484848],
    [0.76507277, 0.92626263, 0.76507277, 0.71666667],
    [0.72077922, 0.85511983, 0.72077922, 0.84197531],
    [0.72619048, 0.8959596, 0.72619048, 0.80965909],
    [0.80451128, 0.81410256, 0.80451128, 0.84266667],
    [0.81547619, 0.88323917, 0.81547619, 0.7725]
]

transposed_data_mi = list(zip(*data_mi))

# Plotting
plt.boxplot(transposed_data_mi, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('MI')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1.0)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 'norm' data
data_norm = [
    [0.90926641, 0.94196429, 0.90926641, 0.97333333],
    [0.98190045, 0.95294118, 0.98190045, 0.86531987],
    [0.95841996, 0.94689542, 0.95841996, 0.95925926],
    [0.91239316, 0.95938375, 0.91239316, 0.94],
    [0.90037594, 0.95776772, 0.90037594, 0.96428571],
    [0.93076923, 0.9515873, 0.93076923, 0.87741935],
    [0.97692308, 0.95210084, 0.97692308, 0.95],
    [0.92857143, 0.97875458, 0.92857143, 0.91111111],
    [0.875, 0.92673993, 0.875, 0.93],
    [0.94429708, 0.96104928, 0.94429708, 0.87301587],
    [0.87941788, 0.96078431, 0.87941788, 0.91818182],
    [0.96410256, 0.97134238, 0.96410256, 0.94230769],
    [0.94540943, 0.95675676, 0.94540943, 0.95185185],
    [0.94871795, 0.99622926, 0.94871795, 0.93928571],
    [0.94017094, 0.96638655, 0.94017094, 0.98148148],
    [0.91269841, 0.95413534, 0.91269841, 0.87575758],
    [1, 0.95873016, 1, 0.88888889],
    [0.97619048, 0.91503759, 0.97619048, 0.96071429],
    [0.93984962, 0.96078431, 0.93984962, 0.92],
    [0.93762994, 0.92134063, 0.93762994, 0.96774194]
]

transposed_data_norm = list(zip(*data_norm))

# Plotting
plt.boxplot(transposed_data_norm, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('norm')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.8, 1.05)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 'STTC' data
data_sttc = [
    [0.85185185, 0.64672365, 0.85185185, 0.66071429],
    [0.74904215, 0.7974026, 0.74904215, 0.79722222],
    [0.73611111, 0.88265306, 0.73611111, 0.80448718],
    [0.71505376, 0.8156682, 0.71505376, 0.74475524],
    [0.73366013, 0.80508475, 0.73366013, 0.58153846],
    [0.65789474, 0.77320955, 0.65789474, 0.78021978],
    [0.63111111, 0.77402597, 0.63111111, 0.63095238],
    [0.64522417, 0.81428571, 0.64522417, 0.68489583],
    [0.64814815, 0.83984868, 0.64814815, 0.62678063],
    [0.6087963, 0.79573935, 0.6087963, 0.83333333],
    [0.81770833, 0.83827493, 0.81770833, 0.76666667],
    [0.90888889, 0.72760291, 0.90888889, 0.6875],
    [0.69444444, 0.72413793, 0.69444444, 0.65680473],
    [0.66666667, 0.78934625, 0.66666667, 0.56410256],
    [0.89247312, 0.82987013, 0.89247312, 0.66025641],
    [0.77083333, 0.81840194, 0.77083333, 0.78205128],
    [0.88888889, 0.90450928, 0.88888889, 0.71666667],
    [0.8125, 0.70823245, 0.8125, 0.79384615],
    [0.79411765, 0.76403061, 0.79411765, 0.74643875],
    [0.875, 0.83673469, 0.875, 0.8543956]
]

transposed_data_sttc = list(zip(*data_sttc))

# Plotting
plt.boxplot(transposed_data_sttc, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('STTC')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

data_average = [
    [0.85112596, 0.822587948, 0.85112596, 0.861674926],
    [0.841615642, 0.85717516, 0.841615642, 0.77085235],
    [0.843072882, 0.921264046, 0.843072882, 0.84417674],
    [0.817898544, 0.859324854, 0.817898544, 0.744778294],
    [0.839393506, 0.884960976, 0.839393506, 0.838414316],
    [0.74030596, 0.901187282, 0.74030596, 0.77621647],
    [0.796189504, 0.89055536, 0.796189504, 0.831092436],
    [0.858235736, 0.916847228, 0.858235736, 0.78410897],
    [0.792784108, 0.843120838, 0.792784108, 0.78138703],
    [0.756029126, 0.882005098, 0.756029126, 0.790517152],
    [0.786647462, 0.891723996, 0.786647462, 0.862945422],
    [0.875437624, 0.863538064, 0.875437624, 0.82831363],
    [0.842942562, 0.905060248, 0.842942562, 0.78096941],
    [0.789481572, 0.862000422, 0.789481572, 0.800552136],
    [0.812335184, 0.899332946, 0.812335184, 0.783479542],
    [0.828212966, 0.863124262, 0.828212966, 0.853078214],
    [0.797741148, 0.90517979, 0.797741148, 0.826446988],
    [0.846428572, 0.8347301, 0.846428572, 0.836058192],
    [0.834822636, 0.830332372, 0.834822636, 0.842095594],
    [0.86570059, 0.876059652, 0.86570059, 0.86157301]
]

transposed_data_average = list(zip(*data_average))

# Plotting
plt.boxplot(transposed_data_average, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('AVERAGE')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.6, 1)
plt.show()


# Male young

In [ ]:

import matplotlib.pyplot as plt

# Your 'CD' data
data_cd = [
    [0.915750916, 0.96059113, 0.966666667, 0.837681159],
    [0.88961039, 0.92021858, 0.839215686, 0.945454545],
    [0.857142857, 0.95321637, 0.972222222, 0.964102564],
    [0.781385281, 0.9382716, 0.91372549, 0.834415584],
    [0.908188586, 0.94615385, 0.779591837, 0.878306878],
    [0.814671815, 0.87953216, 0.916, 0.916149068],
    [0.892307692, 0.91111111, 0.949019608, 0.87755102],
    [0.875, 0.88641975, 0.904, 0.937777778],
    [0.82718894, 0.83615819, 0.933333333, 0.836309524],
    [0.892857143, 0.92105263, 0.945098039, 0.873015873],
    [0.857142857, 0.93333333, 0.808, 0.925925926],
    [0.838709677, 0.91581633, 0.873076923, 0.891625616],
    [0.921428571, 0.84880952, 0.982269504, 0.93125],
    [0.897186147, 0.97142857, 0.976923077, 0.834285714],
    [0.889400922, 0.94787879, 0.839215686, 0.91025641],
    [0.908188586, 0.97142857, 0.932, 0.863636364],
    [0.93452381, 0.85119048, 0.864, 0.927777778],
    [0.930288462, 0.9726776, 1, 0.950310559],
    [0.86965812, 0.95333333, 0.970833333, 0.808571429],
    [0.884169884, 0.97484277, 0.683018868, 0.897515528]
]

transposed_data_cd = list(zip(*data_cd))

# Plotting
plt.boxplot(transposed_data_cd, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('CD')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.75, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Your 'HYP' data
data_hyp = [
    [0.964646465, 0.80859375, 0.713333333, 0.668202765],
    [0.608974359, 0.81617647, 0.955974843, 0.857142857],
    [0.914634146, 0.84960938, 0.715, 0.894957983],
    [0.942307692, 0.86065574, 0.962264151, 0.761111111],
    [0.763888889, 0.91949153, 0.993464052, 0.823529412],
    [0.873015873, 0.9453125, 0.980769231, 0.844827586],
    [0.540625, 0.7807377, 0.65408805, 0.824074074],
    [0.580357143, 0.56352459, 0.576923077, 0.813131313],
    [0.509259259, 0.84469697, 0.823899371, 0.612903226],
    [0.858552632, 0.97420635, 0.660377358, 0.822660099],
    [0.688888889, 0.87873134, 0.634615385, 0.838095238],
    [0.722972973, 0.91733871, 0.969135802, 0.603603604],
    [0.659722222, 0.78174603, 0.793333333, 0.760162602],
    [0.678362573, 0.72222222, 0.938271605, 0.732142857],
    [0.755050505, 0.70362903, 1, 0.795238095],
    [0.71875, 0.70634921, 0.745098039, 0.537634409],
    [0.8125, 0.64786325, 0.666666667, 0.762626263],
    [0.689189189, 0.95772059, 1, 0.88172043],
    [0.679878049, 0.81481481, 0.86, 0.65625],
    [0.783068783, 0.875, 0.745454545, 0.865591398]
]

transposed_data_hyp = list(zip(*data_hyp))

# Plotting
plt.boxplot(transposed_data_hyp, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('HYP')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Your 'MI' data
data_mi = [
    [0.803846154, 0.92592593, 0.995833333, 0.695652174],
    [0.846320346, 0.84386973, 0.953333333, 0.803030303],
    [0.918803419, 0.9368984, 0.868055556, 0.761538462],
    [0.781385281, 0.89869281, 0.736666667, 0.723809524],
    [0.831265509, 0.90702948, 0.914285714, 0.651282051],
    [0.791505792, 0.88786008, 0.942176871, 0.693939394],
    [0.861344538, 0.83877996, 0.886666667, 0.901442308],
    [0.623700624, 0.89651416, 0.921768707, 0.885775862],
    [0.820276498, 0.93055556, 0.58, 0.852272727],
    [0.747767857, 0.82389937, 0.826666667, 0.704761905],
    [0.832653061, 0.9137931, 0.979591837, 0.717948718],
    [0.748847926, 0.93482906, 0.95751634, 0.80952381],
    [0.707142857, 0.88574423, 0.936170213, 0.9875],
    [0.773755656, 0.90849673, 0.970588235, 0.652173913],
    [0.90625, 0.93562708, 0.996666667, 0.805],
    [0.678571429, 0.7672956, 0.853741497, 0.581818182],
    [0.607142857, 0.88095238, 0.891156463, 0.85],
    [0.843317972, 0.83429119, 0.944444444, 0.76969697],
    [0.887755102, 0.86647173, 0.861702128, 0.769021739],
    [0.791505792, 0.93194925, 0.929487179, 0.621212121]
]

transposed_data_mi = list(zip(*data_mi))

# Plotting
plt.boxplot(transposed_data_mi, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('MI')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 'norm' data
data_norm = [
    [0.903846154, 0.96439628, 0.961538462, 0.942857143],
    [0.957013575, 0.91806723, 0.933634992, 0.988888889],
    [0.983673469, 0.97451737, 0.887179487, 0.958064516],
    [0.900452489, 0.96302521, 0.933634992, 0.942386831],
    [0.935483871, 0.97767857, 0.969230769, 0.95483871],
    [0.957528958, 0.96671827, 0.935897436, 0.952380952],
    [0.936263736, 0.98067227, 0.880844646, 0.846875],
    [0.95010395, 0.94537815, 0.915064103, 0.937142857],
    [0.970046083, 0.98382353, 0.90346908, 0.907142857],
    [0.918414918, 0.9554849, 0.954751131, 0.917695473],
    [0.927350427, 0.90285714, 0.947115385, 0.990322581],
    [0.879807692, 0.95428571, 0.877492877, 0.937908497],
    [0.947890819, 0.94038156, 0.956043956, 0.897297297],
    [0.906926407, 0.96507937, 0.927350427, 0.944827586],
    [0.966346154, 0.9754902, 0.97586727, 0.909677419],
    [0.925558313, 0.94833068, 0.942307692, 0.888888889],
    [0.968814969, 0.94285714, 0.956730769, 0.955172414],
    [0.949308756, 0.94773519, 0.972934473, 0.955555556],
    [0.930612245, 0.97071429, 0.93040293, 0.903703704],
    [0.921052632, 0.94636678, 0.928475034, 0.966666667]
]

transposed_data_norm = list(zip(*data_norm))

# Plotting
plt.boxplot(transposed_data_norm, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('norm')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.8, 1.0)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 'STTC' data
data_sttc = [
    [0.703174603, 0.7179803, 0.863636364, 0.788461538],
    [0.779693487, 0.86290323, 0.787234043, 0.77],
    [0.797491039, 0.75738916, 0.83423913, 0.816091954],
    [0.879310345, 0.82727273, 0.969267139, 0.839464883],
    [0.784188034, 0.76780627, 0.903703704, 0.724137931],
    [0.831649832, 0.91995074, 0.857487923, 0.730769231],
    [0.701851852, 0.74725275, 0.9375, 0.702917772],
    [0.732638889, 0.75194805, 0.82712766, 0.579326923],
    [0.664609053, 0.84993695, 0.895833333, 0.667692308],
    [0.768031189, 0.87468672, 0.841145833, 0.625],
    [0.85483871, 0.69906323, 0.909574468, 0.771978022],
    [0.561728395, 0.76530612, 0.953703704, 0.787179487],
    [0.737179487, 0.70802005, 0.792929293, 0.64479638],
    [0.616541353, 0.73182957, 0.923611111, 0.721893491],
    [0.880658436, 0.84438776, 0.872340426, 0.714285714],
    [0.739316239, 0.89097744, 0.880319149, 0.586538462],
    [0.796875, 0.71785714, 0.928191489, 0.789940828],
    [0.742798354, 0.92626728, 0.956018519, 0.656666667],
    [0.840501792, 0.81967213, 0.902777778, 0.660493827],
    [0.769360269, 0.75804196, 0.918367347, 0.673076923]
]

transposed_data_sttc = list(zip(*data_sttc))

# Plotting
plt.boxplot(transposed_data_sttc, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('STTC')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1.0)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 'AVERAGE' data
data_average = [
    [0.858252858, 0.875497478, 0.900201632, 0.786570956],
    [0.816322431, 0.872247048, 0.893878579, 0.872903319],
    [0.894348986, 0.894326136, 0.855339279, 0.878951096],
    [0.856968218, 0.897583618, 0.903111688, 0.820237587],
    [0.844602978, 0.90363194, 0.912055215, 0.806418996],
    [0.853674454, 0.91987475, 0.926466292, 0.827613246],
    [0.786478564, 0.851710758, 0.861623794, 0.830572035],
    [0.752360121, 0.80875694, 0.828976709, 0.830630947],
    [0.758275967, 0.88903424, 0.827307024, 0.775264128],
    [0.837124748, 0.909865994, 0.845607806, 0.78862667],
    [0.832174789, 0.865555628, 0.855779415, 0.848854097],
    [0.750413333, 0.897515186, 0.926185129, 0.805968203],
    [0.794672791, 0.832940278, 0.89214926, 0.844201256],
    [0.774554427, 0.859811292, 0.947348891, 0.777064712],
    [0.879541203, 0.881402572, 0.93681801, 0.826891528],
    [0.794076913, 0.8568763, 0.870693275, 0.691703261],
    [0.823971327, 0.808144078, 0.861349078, 0.857103457],
    [0.830980547, 0.92773837, 0.974679487, 0.842790036],
    [0.841681061, 0.885001258, 0.905143234, 0.75960814],
    [0.829831472, 0.897240152, 0.840960595, 0.804812527]
]

transposed_data_average = list(zip(*data_average))

# Plotting
plt.boxplot(transposed_data_average, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('AVERAGE')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.6, 1.0)
plt.show()


# Female young

In [ ]:
import matplotlib.pyplot as plt

# Your new data
data = [
    [0.89473684, 0.895555556, 0.8627451, 0.922222222],
    [0.82488479, 0.959322034, 0.96153846, 0.78974359],
    [0.93452381, 0.92568306, 0.7375, 0.946428571],
    [0.85221675, 0.976608187, 1, 0.82962963],
    [0.90756303, 0.938011696, 0.94117647, 0.9],
    [0.91666667, 0.913095238, 0.85882353, 0.952380952],
    [0.8531746, 0.962406015, 0.94117647, 0.92364532],
    [0.96270396, 0.936, 0.9875, 0.87654321],
    [0.92647059, 0.888888889, 0.8, 0.880952381],
    [0.89495798, 0.928301887, 0.83265306, 0.928571429],
    [0.93526786, 0.919298246, 0.94117647, 0.948148148],
    [0.84615385, 0.917857143, 0.91923077, 0.975274725],
    [0.81978022, 0.940350877, 0.94166667, 0.818666667],
    [0.83484163, 0.890123457, 0.94166667, 0.878787879],
    [0.91474654, 0.919480519, 0.95217391, 0.900621118],
    [0.78333333, 0.93015873, 0.92156863, 0.903940887],
    [0.88461538, 0.833939394, 1, 0.843478261],
    [0.81274131, 0.934482759, 0.67169811, 0.909090909],
    [0.95752896, 0.967857143, 0.97142857, 0.880102041],
    [0.83571429, 0.842592593, 0.87843137, 0.891156463]
]

transposed_data = list(zip(*data))

# Plotting
plt.boxplot(transposed_data, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False)
plt.title('CD')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Your new data
data = [
    [0.94056848, 0.992537313, 0.69811321, 0.46875],
    [0.5462963, 0.746212121, 0.64242424, 0.923809524],
    [0.62872629, 0.966832504, 1, 0.689795918],
    [0.88562092, 0.951171875, 1, 0.75],
    [0.721875, 0.705078125, 0.77358491, 0.742857143],
    [0.77460317, 0.900793651, 0.83018868, 0.752873563],
    [0.8401084, 0.702380952, 0.80769231, 0.895424837],
    [0.53618421, 0.752192982, 0.5, 0.777777778],
    [0.88125, 0.91015625, 0.87272727, 0.609243697],
    [0.815625, 0.860416667, 1, 0.785714286],
    [0.80921053, 0.84375, 0.94339623, 0.72],
    [0.89430894, 0.543560606, 1, 0.853046595],
    [0.79487179, 0.66015625, 0.78666667, 0.897058824],
    [0.68421053, 0.81352459, 0.76, 0.757936508],
    [0.56756757, 0.842213115, 0.95138889, 0.8],
    [0.55555556, 0.953571429, 0.8490566, 0.689189189],
    [0.62195122, 0.77016129, 0.67283951, 0.869791667],
    [0.81481481, 0.651923077, 0.92727273, 0.926108374],
    [0.43023256, 0.803571429, 0.67973856, 0.779591837],
    [0.82638889, 0.913370998, 0.75, 0.793103448]
]

transposed_data = list(zip(*data))

# Plotting
plt.boxplot(transposed_data, labels=["Male old", "Female young", "Male young", "Female old"],showfliers=False,showmeans=True)
plt.title('HYP')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.4, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Your MI data
data_mi = [
    [0.83082707, 0.914807302, 0.94666667, 0.701086957],
    [0.81009615, 0.927579365, 0.93589744, 0.9525],
    [0.78968254, 0.864942529, 0.92198582, 0.836538462],
    [0.8025641, 0.900205761, 0.9, 0.713942308],
    [0.87472527, 0.826131687, 0.85666667, 0.921212121],
    [0.87619048, 0.85620915, 0.86666667, 0.865131579],
    [0.86111111, 0.831236897, 0.69333333, 0.847619048],
    [0.84375, 0.853427896, 0.81914894, 0.780246914],
    [0.91596639, 0.919753086, 0.92628205, 0.794871795],
    [0.66806723, 0.924444444, 0.94791667, 0.657575758],
    [0.72321429, 0.843850267, 0.99, 0.766666667],
    [0.96230159, 0.85218254, 1, 0.741333333],
    [0.66813187, 0.931069959, 0.91134752, 0.861333333],
    [0.69480519, 0.881263617, 0.91666667, 0.743434343],
    [0.70967742, 0.856334842, 0.84347826, 0.760606061],
    [0.76190476, 0.906481481, 0.85666667, 0.798611111],
    [0.64102564, 0.88034188, 0.95751634, 0.684057971],
    [0.74517375, 0.951515152, 0.96474359, 0.653968254],
    [0.82186235, 0.873165618, 0.92013889, 0.733333333],
    [0.83571429, 0.914648212, 0.71333333, 0.64]
]

transposed_data_mi = list(zip(*data_mi))

# Plotting
plt.boxplot(transposed_data_mi, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('MI')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Your 'norm' data
data_norm = [
    [0.93491124, 0.970714286, 0.95475113, 0.968965517],
    [0.97465438, 0.933333333, 0.91902834, 0.9375],
    [0.9043659, 0.904761905, 0.83516484, 0.90625],
    [0.92051282, 0.975232198, 0.8959276, 0.94375],
    [0.92527473, 0.915057915, 0.92911011, 0.981481481],
    [0.92059553, 0.97217806, 0.91251885, 0.996],
    [0.92857143, 0.957869634, 0.92609351, 0.939393939],
    [0.8997669, 0.951612903, 0.94505495, 0.94375],
    [0.94065934, 0.918146718, 0.94601889, 0.858064516],
    [0.87394958, 0.982698962, 0.95042735, 0.892592593],
    [0.93939394, 0.934362934, 0.92911011, 0.981481481],
    [0.97222222, 0.936029412, 0.96296296, 0.943333333],
    [0.94957983, 0.972200772, 0.93589744, 0.9],
    [0.95475113, 0.974789916, 0.93589744, 0.849002849],
    [1, 0.91512605, 0.96153846, 0.944444444],
    [0.94789082, 0.96013289, 0.95625943, 0.93030303],
    [0.93162393, 0.960784314, 0.95726496, 0.969348659],
    [0.9034749, 0.993233083, 0.96221323, 0.969230769],
    [0.96761134, 0.967460317, 0.96752137, 0.940625],
    [0.97270471, 0.978373702, 0.90648567, 0.931623932]
]

transposed_data_norm = list(zip(*data_norm))

# Plotting
plt.boxplot(transposed_data_norm, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('norm')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.8, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Your 'STTC' data
data_sttc = [
    [0.6879085, 0.777517564, 0.89834515, 0.692307692],
    [0.79423868, 0.757142857, 0.89569161, 0.785714286],
    [0.79513889, 0.843317972, 0.90555556, 0.694444444],
    [0.53111111, 0.853976532, 0.90104167, 0.763888889],
    [0.84074074, 0.75862069, 0.99054374, 0.556666667],
    [0.65384615, 0.695488722, 0.93489583, 0.601449275],
    [0.58333333, 0.788220551, 0.95035461, 0.805128205],
    [0.77380952, 0.643491124, 0.78888889, 0.856763926],
    [0.76481481, 0.777093596, 0.82086168, 0.700549451],
    [0.73518519, 0.887566138, 0.8691358, 0.724358974],
    [0.67063492, 0.815270936, 0.87239583, 0.714285714],
    [0.83506944, 0.866666667, 0.94675926, 0.695156695],
    [0.86296296, 0.846153846, 0.81111111, 0.698005698],
    [0.86015326, 0.716883117, 0.93939394, 0.826373626],
    [0.72222222, 0.77012987, 0.80813953, 0.63],
    [0.8, 0.881696429, 0.86524823, 0.61827957],
    [0.75985663, 0.845663265, 0.83928571, 0.716923077],
    [0.78619529, 0.930992736, 0.9725, 0.7090301],
    [0.82828283, 0.74801061, 0.77777778, 0.801061008],
    [0.81684211, 0.71031746, 0.94326241, 0.692307692]
]

transposed_data_sttc = list(zip(*data_sttc))

# Plotting
plt.boxplot(transposed_data_sttc, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('STTC')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Your 'AVERAGE' data
data_avg = [
    [0.857790426, 0.910226404, 0.872124252, 0.750666478],
    [0.79003406, 0.864717942, 0.870916018, 0.87785348],
    [0.810487486, 0.901107594, 0.880041244, 0.814691479],
    [0.79840514, 0.931438911, 0.939393854, 0.800242165],
    [0.854035754, 0.828580023, 0.89821638, 0.820443482],
    [0.8283804, 0.867552964, 0.880618712, 0.833567074],
    [0.813259774, 0.84842281, 0.863730046, 0.88224227],
    [0.803242918, 0.827344981, 0.808118556, 0.847016365],
    [0.885832226, 0.882807708, 0.873177978, 0.768736368],
    [0.797556996, 0.916685619, 0.920026576, 0.797762608],
    [0.815544308, 0.871306477, 0.935215728, 0.826116402],
    [0.902011208, 0.823259273, 0.965790598, 0.841628936],
    [0.819065334, 0.869986341, 0.877337882, 0.835012904],
    [0.805752348, 0.855316939, 0.898724944, 0.811107041],
    [0.78284275, 0.860656879, 0.90334381, 0.807134325],
    [0.769736894, 0.926408192, 0.889759912, 0.788064757],
    [0.76781456, 0.858178029, 0.885381304, 0.816719927],
    [0.812480012, 0.892429361, 0.899685532, 0.833485681],
    [0.801103608, 0.872013023, 0.863321034, 0.826942644],
    [0.857472858, 0.871860593, 0.838302556, 0.789638307]
]

transposed_data_avg = list(zip(*data_avg))

# Plotting
plt.boxplot(transposed_data_avg, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False, showmeans=True)
plt.title('AVERAGE')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.7, 1.0)
plt.show()


# Male old

In [ ]:
import matplotlib.pyplot as plt

# Your data
data = [
    [0.880494505, 0.89382716, 0.943333333, 0.929824561],
    [0.770909091, 0.842857143, 0.898850575, 0.93452381],
    [0.925, 0.87037037, 0.716666667, 0.912],
    [0.896855346, 0.879487179, 0.808270677, 0.949333333],
    [0.85, 0.841269841, 0.916666667, 0.918719212],
    [0.842528736, 0.863888889, 0.829239766, 0.925925926],
    [0.89005848, 0.864795918, 0.883870968, 0.812169312],
    [0.918886199, 0.907246377, 0.827380952, 0.822222222],
    [0.811111111, 0.740601504, 0.846327684, 0.936],
    [0.784313725, 0.751428571, 0.809195402, 0.933333333],
    [0.833333333, 0.830049261, 0.902380952, 0.827777778],
    [0.839344262, 0.905376344, 0.884097035, 0.775757576],
    [0.810344828, 0.805194805, 0.807397959, 0.905714286],
    [0.865497076, 0.873376623, 0.800242131, 0.866666667],
    [0.891705069, 0.87797619, 0.916384181, 0.935960591],
    [0.845555556, 0.895652174, 0.909195402, 0.81292517],
    [0.947311828, 0.824, 0.778947368, 0.962585034],
    [0.833766234, 0.795566502, 0.93452381, 0.857142857],
    [0.938011696, 0.844444444, 0.812578616, 0.933673469],
    [0.808187135, 0.805333333, 0.92345679, 0.921182266]
]

transposed_data = list(zip(*data))

# Plotting
plt.boxplot(transposed_data, labels=["Male old", "Female young", "Male young", "Female old"])
plt.title('CD')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# New data
data = [
    [0.952586207, 0.838888889, 0.850746269, 0.791666667],
    [0.889112903, 0.954545455, 0.921153846, 0.75],
    [0.865079365, 0.766666667, 0.96031746, 0.794117647],
    [0.808333333, 0.971428571, 0.942460317, 0.656862745],
    [0.886904762, 0.99047619, 0.917624521, 0.869047619],
    [0.890384615, 0.833333333, 0.775390625, 0.764705882],
    [0.880859375, 0.819444444, 0.93115942, 0.731092437],
    [0.909615385, 0.802083333, 0.906746032, 0.807407407],
    [0.930327869, 0.913580247, 0.814393939, 0.774509804],
    [0.836206897, 0.939393939, 0.934615385, 0.853061224],
    [0.956349206, 0.838461538, 0.972222222, 0.919642857],
    [0.952205882, 0.695833333, 0.898305085, 0.919354839],
    [0.991319444, 0.917695473, 0.943548387, 0.797979798],
    [0.96875, 0.944444444, 0.913461538, 0.804761905],
    [0.854779412, 0.822916667, 0.883760684, 0.738738739],
    [0.979477612, 0.885416667, 0.898076923, 0.423469388],
    [0.762681159, 0.906862745, 0.955078125, 0.752873563],
    [0.875, 0.80952381, 0.869047619, 0.711711712],
    [0.902998236, 0.811111111, 0.866666667, 0.93877551],
    [0.923828125, 0.977142857, 0.918032787, 0.642857143]
]

transposed_data = list(zip(*data))

# Plotting
plt.boxplot(transposed_data, labels=["Male old", "Female young", "Male young", "Female old"] , showfliers=False)
plt.title('HYP')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# New data
data = [
    [0.867947179, 0.883950617, 0.861598441, 0.775438596],
    [0.892341842, 0.872282609, 0.827272727, 0.579545455],
    [0.908805031, 0.866666667, 0.732704403, 0.762666667],
    [0.865555556, 0.75, 0.913943355, 0.917333333],
    [0.881551363, 0.792307692, 0.899092971, 0.759259259],
    [0.778787879, 0.783333333, 0.832304527, 0.682051282],
    [0.87962963, 0.8, 0.791176471, 0.917948718],
    [0.847689076, 0.84057971, 0.875262055, 0.658333333],
    [0.852941176, 0.796296296, 0.848214286, 0.730666667],
    [0.925925926, 0.711111111, 0.832983193, 0.814814815],
    [0.841719078, 0.726190476, 0.86687631, 0.788043478],
    [0.729885057, 0.619354839, 0.908163265, 0.781818182],
    [0.924242424, 0.928125, 0.891025641, 0.701086957],
    [0.819251337, 0.81875, 0.898989899, 0.83030303],
    [0.877394636, 0.75942029, 0.895833333, 0.891203704],
    [0.863547758, 0.809659091, 0.822478992, 0.793333333],
    [0.907721281, 0.824, 0.905349794, 0.7],
    [0.82788671, 0.597619048, 0.835511983, 0.761904762],
    [0.896090535, 0.857142857, 0.947777778, 0.772839506],
    [0.905349794, 0.692708333, 0.961873638, 0.791666667]
]

transposed_data = list(zip(*data))

# Plotting without outliers
plt.boxplot(transposed_data, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False)
plt.title('MI')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# New data
data = [
    [0.97235023, 0.956228956, 0.959110473, 0.937777778],
    [0.94122449, 0.979310345, 0.965413534, 0.961685824],
    [0.926073132, 0.951612903, 0.837301587, 0.94265233],
    [0.94550173, 0.899305556, 0.957936508, 0.93],
    [0.925396825, 0.909677419, 0.94741533, 0.927272727],
    [0.943609023, 0.903448276, 0.914086687, 0.906451613],
    [0.918146718, 0.903125, 0.931972789, 0.929032258],
    [0.93438914, 0.971428571, 0.953895072, 0.927586207],
    [0.852941176, 0.972222222, 0.946323529, 0.92],
    [0.952205882, 0.8, 0.869532428, 0.9125],
    [0.886327504, 0.903030303, 0.912698413, 0.889655172],
    [0.907563025, 0.891666667, 0.935828877, 0.977777778],
    [0.946616541, 0.969230769, 0.892973856, 0.951724138],
    [0.892664093, 0.976923077, 0.909022556, 0.972222222],
    [0.945378151, 0.908045977, 0.928676471, 0.906060606],
    [0.934002869, 0.907142857, 0.886877828, 0.935897436],
    [0.93537415, 0.923333333, 0.942084942, 0.956],
    [0.895798319, 0.895424837, 0.914285714, 0.860606061],
    [0.936532508, 0.969230769, 0.947186147, 0.903125],
    [0.926640927, 0.88, 0.981512605, 0.975757576]
]

transposed_data = list(zip(*data))

# Plotting without outliers
plt.boxplot(transposed_data, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False)
plt.title('NORM')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# New data
data = [
    [0.867924528, 0.938992042, 0.838407494, 0.728937729],
    [0.922193878, 0.885802469, 0.937179487, 0.775641026],
    [0.898496241, 0.813186813, 0.824561404, 0.711309524],
    [0.886713287, 0.870689655, 0.865914787, 0.504273504],
    [0.921052632, 0.862637363, 0.936657682, 0.779487179],
    [0.945520581, 0.781065089, 0.834419817, 0.802197802],
    [0.933497537, 0.74270557, 0.905048077, 0.675824176],
    [0.897094431, 0.887820513, 0.9160401, 0.766272189],
    [0.927272727, 0.96031746, 0.966666667, 0.794871795],
    [0.898351648, 0.784023669, 0.898305085, 0.684350133],
    [0.895989975, 0.905128205, 0.913533835, 0.651234568],
    [0.882488479, 0.794871795, 0.935309973, 0.663461538],
    [0.885897436, 0.913043478, 0.929846939, 0.666666667],
    [0.90482399, 0.822916667, 0.905569007, 0.794871795],
    [0.877880184, 0.875, 0.902380952, 0.731182796],
    [0.843091335, 0.737179487, 0.915254237, 0.597826087],
    [0.842403628, 0.763532764, 0.896551724, 0.797202797],
    [0.857142857, 0.779487179, 0.733082707, 0.68974359],
    [0.878078818, 0.859531773, 0.924475524, 0.75596817],
    [0.875615764, 0.888888889, 0.951948052, 0.706989247]
]

transposed_data = list(zip(*data))

# Plotting without outliers
plt.boxplot(transposed_data, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False)
plt.title('STTC')
plt.ylabel('Values')
# Adjusting y-axis limits
plt.ylim(0.5, 1.1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# New data
data = [
    [0.90826053, 0.902377533, 0.890639202, 0.832729066],
    [0.883156441, 0.906959604, 0.909974034, 0.800279223],
    [0.904690754, 0.853700684, 0.814310304, 0.824549233],
    [0.88059185, 0.874182192, 0.897705129, 0.791560583],
    [0.892981116, 0.879273701, 0.923491434, 0.850757199],
    [0.880166167, 0.833013784, 0.837088285, 0.816266501],
    [0.900438348, 0.826014187, 0.888645545, 0.81321338],
    [0.901534846, 0.881831701, 0.895864842, 0.796364272],
    [0.874918812, 0.876603546, 0.884385221, 0.831209653],
    [0.879400816, 0.797191458, 0.868926299, 0.839611901],
    [0.882743819, 0.840571957, 0.913542346, 0.815270771],
    [0.862297341, 0.781420596, 0.912340847, 0.823633983],
    [0.911684135, 0.906657905, 0.892958556, 0.804634369],
    [0.890197299, 0.887282162, 0.885457026, 0.853765124],
    [0.88942749, 0.848671825, 0.905407124, 0.840629287],
    [0.893135026, 0.847010055, 0.886376676, 0.712690283],
    [0.879098409, 0.848345768, 0.895602391, 0.833732279],
    [0.857918824, 0.775524275, 0.857290366, 0.776221796],
    [0.910342359, 0.868292191, 0.899736946, 0.860876331],
    [0.887924349, 0.848814683, 0.947364774, 0.80769058]
]

transposed_data = list(zip(*data))

# Plotting without outliers and setting y-axis limits
plt.boxplot(transposed_data, labels=["Male old", "Female young", "Male young", "Female old"], showfliers=False)
plt.title('Average')
plt.ylabel('Values')

# Adjusting y-axis limits
plt.ylim(0.7, 1)

plt.show()


In [ ]:
Y_pred_binary = Y_pred.round()

In [ ]:
Y_pred_binary

In [ ]:
Y_test

In [ ]:
import sklearn.metrics as skm

cm = skm.multilabel_confusion_matrix(Y_pred_binary, Y_test)

# for i in cm:
#     total = i.sum()
#     percentage_cm = (i / total) * 100
#     print(percentage_cm)

In [ ]:

print(cm)

In [ ]:
from sklearn.metrics import roc_auc_score

    # True labels (ground truth) and predicted probabilities
for i in range(5):    
    true_labels = Y_test[:,i]
    predicted_probs = Y_pred[:,i]

    # Calculate the AUC score
    auc_score = roc_auc_score(true_labels, predicted_probs)

    print(f"AUC Score {i}: {auc_score}")


In [ ]:
type(cm)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define the new confusion matrices
confusion_matrices = [
    np.array([[964 , 39], [41,63]]),
    np.array([[1038 , 38], [7, 24]]),
    np.array([[926 , 37], [63 , 81]]),
    np.array([[224 , 42], [103 , 738]]),
    np.array([[859, 41], [77, 130]])
]

# Define labels for each confusion matrix
matrix_labels = [
    "CD",
    "HYP",
    "MI",
    "NORM",
    "STTC"
]

# Create subplots for each confusion matrix
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# Plot each confusion matrix with percentages and labels
for i, ax in enumerate(axes):
    cm = confusion_matrices[i]
    # Calculate row percentages
    row_sums = cm.sum(axis=0, keepdims=True)
    percentages = cm / row_sums
    labels = np.array([['{:.2%}'.format(val) for val in row] for row in percentages])
    sns.heatmap(percentages, annot=labels, fmt="", cmap="Blues", ax=ax, cbar=False)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(matrix_labels[i])

fig.suptitle("Confusion Matrices for Male <= 65")

# Save the image as a file (e.g., in PNG format)
plt.savefig("confusion_matrices.png")

# Display the image
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define the new confusion matrices
confusion_matrices = cm

# Define labels for each confusion matrix
matrix_labels = [
    "CD",
    "HYP",
    "MI",
    "NORM",
    "STTC"
]

# Create subplots for each confusion matrix
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# Plot each confusion matrix with percentages and labels
for i, ax in enumerate(axes):
    cm = confusion_matrices[i]
    # Calculate row percentages
    row_sums = cm.sum(axis=0, keepdims=True)
    percentages = cm / row_sums
    labels = np.array([['{:.2%}'.format(val) for val in row] for row in percentages])
    sns.heatmap(percentages, annot=labels, fmt="", cmap="Blues", ax=ax, cbar=False)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(matrix_labels[i])

fig.suptitle("Confusion Matrices for Female > 65")

# Save the image as a file (e.g., in PNG format)
plt.savefig("confusion_matrices.png")

# Display the image
plt.show()


In [ ]:
type(cm[0])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# Define the new confusion matrices
confusion_matrices = [
    np.array([[1074, 63], [62,233]]),
    np.array([[1245 , 63], [25, 99]]),
    np.array([[986 , 41], [91 , 314]]),
    np.array([[626 , 36], [116 , 654]]),
    np.array([[1072, 40], [84, 236]])
]

# Define labels for each confusion matrix
matrix_labels = [
    "CD",
    "HYP",
    "MI",
    "NORM",
    "STTC"
]

# Create subplots for each confusion matrix
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# Plot each confusion matrix with percentages and labels
for i, ax in enumerate(axes):
    cm = confusion_matrices[i]
    # Calculate row percentages
    row_sums = cm.sum(axis=0, keepdims=True)
    percentages = cm / row_sums
    labels = np.array([['{:.2%}'.format(val) for val in row] for row in percentages])
    sns.heatmap(percentages, annot=labels, fmt="", cmap="Blues", ax=ax, cbar=False)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(matrix_labels[i])

fig.suptitle("Confusion Matrices for Female <= 65")

# Save the image as a file (e.g., in PNG format)
plt.savefig("confusion_matrices.png")

# Display the image
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define the new confusion matrices
confusion_matrices = [
    np.array([[622 , 90], [52,185]]),
    np.array([[748 , 92], [31, 78]]),
    np.array([[578 , 62], [97 , 212]]),
    np.array([[569 , 24], [114 , 242]]),
    np.array([[465, 75], [122, 287]])
]

# Define labels for each confusion matrix
matrix_labels = [
    "CD",
    "HYP",
    "MI",
    "NORM",
    "STTC"
]

# Create subplots for each confusion matrix
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# Plot each confusion matrix with percentages and labels
for i, ax in enumerate(axes):
    cm = confusion_matrices[i]
    # Calculate row percentages
    row_sums = cm.sum(axis=0, keepdims=True)
    percentages = cm / row_sums
    labels = np.array([['{:.2%}'.format(val) for val in row] for row in percentages])
    sns.heatmap(percentages, annot=labels, fmt="", cmap="Blues", ax=ax, cbar=False)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(matrix_labels[i])

fig.suptitle("Confusion Matrices for Male > 65")

# Save the image as a file (e.g., in PNG format)
plt.savefig("confusion_matrices.png")

# Display the image
plt.show()


In [ ]:
# Female young with Female Old
# AUC Score: 0.9422586470726092
# AUC Score: 0.8837449483610238
# AUC Score: 0.9294454551580082
# AUC Score: 0.9618224057821374
# AUC Score: 0.905939139250922

# Cross test female yougn with male old
# AUC Score: 0.9000701375775559
# AUC Score: 0.8392962319716077
# AUC Score: 0.9001027304676938
# AUC Score: 0.9458547540153458
# AUC Score: 0.

# Femal young with female young
# AUC Score: 0.9468351494099734
# AUC Score: 0.9539613103917567
# AUC Score: 0.9602469038931827
# AUC Score: 0.9648091722332903
# AUC Score: 0.9505823429115892

# Female young and male young
# AUC Score: 0.9285240464344943
# AUC Score: 0.9329834851057262
# AUC Score: 0.9256310945827835
# AUC Score: 0.9370697090880578
# AUC Score: 0.9349165292147749

In [ ]:
# [[86.269196  3.432701]
#  [ 4.516712  5.781391]]
# [[93.857272  3.613369]
#  [ 0.542005  1.987353]]
# [[86.449864  5.058717]
#  [ 2.890696  5.600723]]
# [[18.247516  3.252033]
#  [11.29178  67.208672]]
# [[78.319783  4.878049]
#  [ 6.233062 10.569106]]

In [ ]:
# male old

# [[[3192  340]
#   [ 256  957]]

#  [[3819  385]
#   [  99  442]]

#  [[2901  344]
#   [ 375 1125]]

#  [[2950  126]
#   [ 460 1209]]

#  [[2423  286]
#   [ 557 1479]]]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define the new confusion matrices
confusion_matrices = [
    np.array([[1507 , 148], [101,467]]),
    np.array([[1866, 150], [48, 159]]),
    np.array([[1375 , 124], [171 , 553]]),
    np.array([[1117 , 51], [230 , 825]]),
    np.array([[1561, 94], [164, 404]])
]

# Define labels for each confusion matrix
matrix_labels = [
    "CD",
    "HYP",
    "MI",
    "NORM",
    "STTC"
]

# Create subplots for each confusion matrix
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# Plot each confusion matrix with percentages and labels
for i, ax in enumerate(axes):
    cm = confusion_matrices[i]
    # Calculate row percentages
    row_sums = cm.sum(axis=0, keepdims=True)
    percentages = cm / row_sums
    labels = np.array([['{:.2%}'.format(val) for val in row] for row in percentages])
    sns.heatmap(percentages, annot=labels, fmt="", cmap="Blues", ax=ax, cbar=False)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(matrix_labels[i])

fig.suptitle("Confusion Matrices for Female")

# Save the image as a file (e.g., in PNG format)
plt.savefig("confusion_matrices.png")

# Display the image
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define the new confusion matrices
confusion_matrices = [
    np.array([[1621 , 117], [75,243]]),
    np.array([[1781, 108], [46, 121]]),
    np.array([[1503 , 89], [163 , 301]]),
    np.array([[797 , 73], [209 , 977]]),
    np.array([[1329, 96], [184, 447]])
]

# Define labels for each confusion matrix
matrix_labels = [
    "CD",
    "HYP",
    "MI",
    "NORM",
    "STTC"
]

# Create subplots for each confusion matrix
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# Plot each confusion matrix with percentages and labels
for i, ax in enumerate(axes):
    cm = confusion_matrices[i]
    # Calculate row percentages
    row_sums = cm.sum(axis=0, keepdims=True)
    percentages = cm / row_sums
    labels = np.array([['{:.2%}'.format(val) for val in row] for row in percentages])
    sns.heatmap(percentages, annot=labels, fmt="", cmap="Blues", ax=ax, cbar=False)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(matrix_labels[i])

fig.suptitle("Confusion Matrices for male")

# Save the image as a file (e.g., in PNG format)
plt.savefig("confusion_matrices.png")

# Display the image
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define the new confusion matrices
confusion_matrices = [
    np.array([[5364, 478], [284, 1034]]),
    np.array([[6167, 479], [145, 369]]),
    np.array([[4964, 465], [346, 1385]]),
    np.array([[2838, 172], [879, 3271]]),
    np.array([[5313, 316], [489, 1042]])
]

# Define labels for each confusion matrix
matrix_labels = [
    "CD",
    "HYP",
    "MI",
    "NORM",
    "STTC"
]

# Create subplots for each confusion matrix
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# Plot each confusion matrix with percentages and labels
for i, ax in enumerate(axes):
    cm = confusion_matrices[i]
    # Calculate row percentages
    row_sums = cm.sum(axis=0, keepdims=True)
    percentages = cm / row_sums
    labels = np.array([['{:.2%}'.format(val) for val in row] for row in percentages])
    sns.heatmap(percentages, annot=labels, fmt="", cmap="Blues", ax=ax, cbar=False)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(matrix_labels[i])

fig.suptitle("Confusion Matrices for Female")
plt.show()




# Calculate and display percentages
for i, cm in enumerate(confusion_matrices):
    tn, fp, fn, tp = cm.ravel()
    total_instances = tn + fp + fn + tp
    tn_percent = (tn / total_instances) * 100
    fp_percent = (fp / total_instances) * 100
    fn_percent = (fn / total_instances) * 100
    tp_percent = (tp / total_instances) * 100

    print(f"Class {i + 1}:")
    print(f"True Negatives: {tn_percent}%")
    print(f"False Positives: {fp_percent}%")
    print(f"False Negatives: {fn_percent}%")
    print(f"True Positives: {tp_percent}%")
    print()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define the new confusion matrices
confusion_matrices = [
    np.array([[5364, 478], [284, 1034]]),
    np.array([[6167, 479], [145, 369]]),
    np.array([[4964, 465], [346, 1385]]),
    np.array([[2838, 172], [879, 3271]]),
    np.array([[5313, 316], [489, 1042]])
]

# Define labels for each confusion matrix
matrix_labels = [
    "CD",
    "HYP",
    "MI",
    "NORM",
    "STTC"
]

# Create subplots for each confusion matrix
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# Plot each confusion matrix with percentages and labels
for i, ax in enumerate(axes):
    cm = confusion_matrices[i]
    # Calculate row percentages
    row_sums = cm.sum(axis=0, keepdims=True)
    percentages = cm / row_sums
    labels = np.array([['{:.2%}'.format(val) for val in row] for row in percentages])
    sns.heatmap(percentages, annot=labels, fmt="", cmap="Blues", ax=ax, cbar=False)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(matrix_labels[i])

fig.suptitle("Confusion Matrices for female <= 65")

# Save the image as a file (e.g., in PNG format)
plt.savefig("confusion_matrices.png")

# Display the image
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define the new confusion matrices
confusion_matrices = [
    np.array([[4814, 222], [116, 380]]),
    np.array([[5200, 183], [30, 119]]),
    np.array([[4743, 196], [196, 397]]),
    np.array([[1219, 138], [520, 3655]]),
    np.array([[4295, 173], [333, 731]])
]

# Define labels for each confusion matrix
matrix_labels = [
    "CD",
    "HYP",
    "MI",
    "NORM",
    "STTC"
]

# Create subplots for each confusion matrix
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# Plot each confusion matrix with percentages and labels
for i, ax in enumerate(axes):
    cm = confusion_matrices[i]
    # Calculate row percentages
    row_sums = cm.sum(axis=0, keepdims=True)
    percentages = cm / row_sums
    labels = np.array([['{:.2%}'.format(val) for val in row] for row in percentages])
    sns.heatmap(percentages, annot=labels, fmt="", cmap="Blues", ax=ax, cbar=False)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(matrix_labels[i])

fig.suptitle("Confusion Matrices for male <= 65")

plt.savefig("original.png")
plt.show()


In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # Define your confusion matrices
# confusion_matrices = [
#     np.array([[2210, 338], [254, 1149]]),
#     np.array([[3202, 376], [77, 296]]),
#     np.array([[2067, 370], [327, 1187]]),
#     np.array([[2602, 101], [406, 842]]),
#     np.array([[2308, 232], [435, 976]])
# ]
# # 
# # Define labels for each confusion matrix
# matrix_labels = [
#     "CD",
#     "HYP",
#     "MI",
#     "NORM",
#     "STTC"
# ]

# # Create subplots for each confusion matrix
# fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# # Plot each confusion matrix with percentages and labels
# for i, ax in enumerate(axes):
#     cm = confusion_matrices[i]
#     # Calculate row percentages
#     row_sums = cm.sum(axis=1, keepdims=True)
#     percentages = cm / row_sums
#     labels = np.array([['{:.2%}'.format(val) for val in row] for row in percentages])
#     sns.heatmap(percentages, annot=labels, fmt="", cmap="Blues", ax=ax, cbar=False)
#     ax.set_xlabel('Predicted')
#     ax.set_ylabel('Actual')
#     ax.set_title(matrix_labels[i])

# fig.suptitle("Confusion Matrices for female > 65")
# plt.show()


In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # Define the new confusion matrices
# confusion_matrices = [
#     np.array([[3192, 340], [256, 957]]),
#     np.array([[3819, 385], [99, 442]]),
#     np.array([[2901, 344], [375, 1125]]),
#     np.array([[2950, 126], [460, 1209]]),
#     np.array([[2423, 286], [557, 1479]])
# ]

# # Define labels for each confusion matrix
# matrix_labels = [
#     "CD",
#     "HYP",
#     "MI",
#     "NORM",
#     "STTC"
# ]

# # Create subplots for each confusion matrix
# fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# # Plot each confusion matrix with percentages and labels
# for i, ax in enumerate(axes):
#     cm = confusion_matrices[i]
#     # Calculate row percentages
#     row_sums = cm.sum(axis=1, keepdims=True)
#     percentages = cm / row_sums
#     labels = np.array([['{:.2%}'.format(val) for val in row] for row in percentages])
#     sns.heatmap(percentages, annot=labels, fmt="", cmap="Blues", ax=ax, cbar=False)
#     ax.set_xlabel('Predicted')
#     ax.set_ylabel('Actual')
#     ax.set_title(matrix_labels[i])
# fig.suptitle("Confusion Matrices for male >  65")
# plt.show()


In [ ]:
# X = load_raw_data(df_female_young,sampling_rate,path)

In [ ]:
# Y = np.array(df_female_young[['CD','HYP','MI','NORM','STTC']])
# # 